In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

In [3]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow.keras.layers import TimeDistributed
tf.keras.backend.clear_session()
from tensorflow.keras.layers import Input, Softmax, RNN, Dense, Embedding, LSTM
from tensorflow.keras.models import Model
import warnings
warnings.filterwarnings('ignore')

In [ ]:
nRowsRead = 50000
df = pd.read_csv('/content/drive/MyDrive/archive (1)/emails.csv', nrows = nRowsRead)

In [ ]:
df.head(5)

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [ ]:
df.shape

(50000, 2)

In [ ]:
import email

In [ ]:
def parse_func(x):
    b = email.message_from_string(x)
    body = ''
    if b.is_multipart():
        for payload in b.get_payload():
            body = body+ ' '
            body+=payload.get_payload()
    else:
        body+=b.get_payload()
    return body
def parse_func_subj(x):
    b = email.message_from_string(x)
    if b.get('Subject'):
        subject = b.get('Subject')
    else:
        subject = '-'
    return subject
df['Body'] = df['message'].apply(parse_func)
df['Subject'] = df['message'].apply(parse_func_subj)

In [ ]:
df.head()

,file,message,Body,Subject
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast\n\n,-
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes the...,Re:
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!,Re: test
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy,\n\n Can you send me a schedule of the s...",-
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.,Re: Hello


In [ ]:
pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 10.3 MB/s eta 0:00:00


In [ ]:
import contractions

In [ ]:
import re
body = []
for text in df['Body']:
    text = contractions.fix(text)
    text = re.sub(r"\n|\t", ' ', text)
    text = re.sub("[^a-zA-Z\s.]+",' ',text)
    text = re.sub("\s+", ' ', text)
    body.append(text)
df['Processed_Text'] = body
df

,file,message,Body,Subject,Processed_Text
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast\n\n,-,Here is our forecast
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes the...,Re:,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!,Re: test,test successful. way to go
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy,\n\n Can you send me a schedule of the s...",-,Randy Can you send me a schedule of the salary...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.,Re: Hello,Let us shoot for Tuesday at .
...,...,...,...,...,...
49995,cash-m/sent_items/346.,Message-ID: <1815718.1075855358574.JavaMail.ev...,"please file ""Pam Benson"". Thanks. MHC\n\n --...",FW: Pam Benson,please file Pam Benson . Thanks. MHC Original ...
49996,cash-m/sent_items/347.,Message-ID: <15801367.1075855358596.JavaMail.e...,please file in Diomedes' file. Thanks. MHC\n...,FW: ECI,please file in Diomedes file. Thanks. MHC Orig...
49997,cash-m/sent_items/348.,Message-ID: <23815150.1075855358619.JavaMail.e...,fine with me. mhc\n\n-----Original Message---...,RE: In re Event Resources,fine with me. mhc Original Message From Davis ...
49998,cash-m/sent_items/349.,Message-ID: <23974470.1075855358641.JavaMail.e...,You need more than the 840 for this -- you als...,RE: List of People,You need more than the for this you also need ...


In [ ]:
body = []
for text in df['Subject']:
  text = text.lower()
  text = re.sub('re:','',text)
  text = contractions.fix(text)
  text = re.sub(r"\n|\t", ' ', text)
  text = re.sub("[^a-zA-Z\s.]+",' ',text)
  text = re.sub("\s+", ' ', text)
  body.append(text.strip())
df['Processed_Subject'] = body
df

,file,message,Body,Subject,Processed_Text,Processed_Subject
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast\n\n,-,Here is our forecast,
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes the...,Re:,Traveling to have a business meeting takes the...,
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!,Re: test,test successful. way to go,test
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy,\n\n Can you send me a schedule of the s...",-,Randy Can you send me a schedule of the salary...,
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.,Re: Hello,Let us shoot for Tuesday at .,hello
...,...,...,...,...,...,...
49995,cash-m/sent_items/346.,Message-ID: <1815718.1075855358574.JavaMail.ev...,"please file ""Pam Benson"". Thanks. MHC\n\n --...",FW: Pam Benson,please file Pam Benson . Thanks. MHC Original ...,fw pam benson
49996,cash-m/sent_items/347.,Message-ID: <15801367.1075855358596.JavaMail.e...,please file in Diomedes' file. Thanks. MHC\n...,FW: ECI,please file in Diomedes file. Thanks. MHC Orig...,fw eci
49997,cash-m/sent_items/348.,Message-ID: <23815150.1075855358619.JavaMail.e...,fine with me. mhc\n\n-----Original Message---...,RE: In re Event Resources,fine with me. mhc Original Message From Davis ...,in re event resources
49998,cash-m/sent_items/349.,Message-ID: <23974470.1075855358641.JavaMail.e...,You need more than the 840 for this -- you als...,RE: List of People,You need more than the for this you also need ...,list of people


In [ ]:
df['Subject'] = df['Subject'].str.replace(r'^(Re|RE):\s*', '', regex=True, flags=re.IGNORECASE)

In [ ]:
remove=[]
for i in df.index:
    text = df.loc[i]['Body']
    if ('Original Message' in text) or ('Forwarded by' in text):
        remove.append(i)
df.drop(remove,axis=0,inplace=True)
df.reset_index(drop=True,inplace=True)

In [ ]:
df.shape

(34763, 6)

In [ ]:
df[df['Processed_Text'].str.startswith('Original Message')]

,file,message,Body,Subject,Processed_Text,Processed_Subject


In [ ]:
sentences = pd.Series([len(str(x).split('.')) for x in df['Processed_Text']])
df['no_of_sent'] = sentences

In [ ]:
df.head()

,file,message,Body,Subject,Processed_Text,Processed_Subject,no_of_sent
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast\n\n,-,Here is our forecast,,1
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes the...,Re:,Traveling to have a business meeting takes the...,,11
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!,Re: test,test successful. way to go,test,2
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy,\n\n Can you send me a schedule of the s...",-,Randy Can you send me a schedule of the salary...,,3
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.,Re: Hello,Let us shoot for Tuesday at .,hello,2


In [ ]:
df_original = df

In [ ]:
#Short Sentences
df = df_original[df_original['no_of_sent']<10]
df.reset_index(drop=True,inplace=True)

In [ ]:
'''Take sentences which have word length in range 5 and 12, split the sentence at each word to prepare inputs and outputs.
To inputs add subject of the mail'''

input = []
output = []
for i in df.index:
  subject = df.loc[i]['Processed_Subject']
  sentences = df.loc[i]['Processed_Text'].split('.')
  for sent in sentences:
    sent = sent.lower()
    noof_words = len(sent.split())
    if noof_words in range(5,13):
      # half = int(noof_words/2)
      words = sent.split()
      for i in range(3,noof_words):
        if i%3==0:
          if noof_words-i>=4:
            input.append(' '.join(subject.split()[:5]+words[:i]))
            output.append(' '.join(words[i:i+4]))
        elif i%3==1:
          if noof_words-i>=3:
            input.append(' '.join(subject.split()[:5]+words[:i]))
            output.append(' '.join(words[i:i+3]))
        else:
          if noof_words-i>=2:
            input.append(' '.join(subject.split()[:5]+words[:i]))
            output.append(' '.join(words[i:i+2]))

In [ ]:
data_short = pd.DataFrame(columns=['input','output'])
data_short['input']=input
data_short['output']=output
data_short

,input,output
0,plus your thoughts,on any changes that
1,plus your thoughts on,any changes that
2,plus your thoughts on any,changes that
3,plus your thoughts on any changes,that need to be
4,plus your thoughts on any changes that,need to be
...,...,...
91060,data request from counterparty i have received an,urgent data request
91061,data request from counterparty i have received...,data request
91062,data request from counterparty i have received...,request from the counterparty
91063,data request from counterparty i have received...,from the counterparty


In [ ]:
#Long Sentences
df = df_original[df_original['no_of_sent']<10]
df.reset_index(drop=True,inplace=True)

In [ ]:
'''Take sentences which have word length in range 10 and 14, split the sentence at each word to prepare inputs and outputs.
To inputs add subject of the mail'''

import random
input = []
output = []
for i in df.index:
  subject = df.loc[i]['Processed_Subject']
  sentences = df.loc[i]['Processed_Text'].split('.')
  for sent in sentences:
    sent = sent.lower()
    noof_words = len(sent.split())
    if noof_words in range(10,15):
      # half = int(noof_words/2)
      words = sent.split()
      for i in range(10,noof_words):
        rn = random.randint(3,6)
        if noof_words-i>=rn:
          input.append(' '.join(subject.split()[:5]+words[:i]))
          output.append(' '.join(words[i:i+rn]))

In [ ]:
data_long = pd.DataFrame(columns=['input','output'])
data_long['input']=input
data_long['output']=output
data_long.head(5)

,input,output
0,com monique sanchez frank ermis john lavorato ...,your help phillip
1,sagewood etc. jeff you would clearly receive a...,on the sagewood
2,western gas market report draft richard compar...,california gas report
3,checkbook and budget you will probably need to...,to an email
4,if we have one year of data we can tell which,will be cheaper


In [ ]:
#Word Breaking
df = df_original[df_original['no_of_sent']<10]
df.reset_index(drop=True,inplace=True)

In [ ]:
'''Take sentences which have word length in range 5 and 12, split the sentence at each word by splitting the word into halves
 by putting first half in input and other half in output. To inputs add subject of the mail'''

input = []
output = []
for i in df.index:
  subject = df.loc[i]['Processed_Subject']
  sentences = df.loc[i]['Processed_Text'].split('.')
  for sent in sentences:
    sent = sent.lower()
    noof_words = len(sent.split())
    if noof_words in range(5,13):
      # half = int(noof_words/2)
      words = sent.split()
      for i in range(3,noof_words):
        if len(words[i])>=5 and noof_words-i>=3:
          for j in range(3,len(words[i])):
            if len(words[i])-j>=2:
              half_word_1 = words[i][:j]
              half_word_2 = words[i][j:]
              input.append(' '.join(subject.split()[:5]+words[:i-1]+[half_word_1]))
              output.append(' '.join([half_word_2]+words[i+1:i+3]))

In [ ]:
data_wordbrk = pd.DataFrame(columns=['input','output'])
data_wordbrk['input']=input
data_wordbrk['output']=output
data_wordbrk.head()

,input,output
0,plus your thoughts on cha,nges that need
1,plus your thoughts on chan,ges that need
2,plus your thoughts on chang,es that need
3,hello greg how eit,her next tuesday
4,hello greg how eith,er next tuesday


In [ ]:
final_data = pd.concat([data_short, data_long, data_wordbrk])
final_data = final_data.drop_duplicates(ignore_index=True)

In [ ]:
new_data = final_data

In [ ]:
final_data.shape

(85412, 2)

In [ ]:
import pickle
outfile = open('/content/drive/MyDrive/final_data.pkl','wb')
pickle.dump(new_data,outfile)
outfile.close()

In [4]:
import pickle
infile = open('/content/drive/MyDrive/final_data.pkl','rb')
new_data = pickle.load(infile)
infile.close()

In [5]:
'''Prepare the data in a format according to Encoder Decoder model'''

new_data['decoder_input'] = '<start> ' + new_data['output'].astype(str)
new_data['decoder_output'] = new_data['output'].astype(str) + ' <end>'
new_data = new_data.drop(['output'], axis=1)
new_data.head()

,input,decoder_input,decoder_output
0,plus your thoughts,<start> on any changes that,on any changes that <end>
1,plus your thoughts on,<start> any changes that,any changes that <end>
2,plus your thoughts on any,<start> changes that,changes that <end>
3,plus your thoughts on any changes,<start> that need to be,that need to be <end>
4,plus your thoughts on any changes that,<start> need to be,need to be <end>


In [6]:
max_len_enc = max([len(x.split()) for x in new_data.input])
max_len_dec = max([len(x.split()) for x in new_data.decoder_input])

In [7]:
max_len_enc, max_len_dec

(16, 5)

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [9]:
'''Tokenize the encoder's vocab and decoder vocab.
In decoder's tokenizer, we have to filter some characters to include ,  as unique tokens'''

#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer_encoder = Tokenizer(oov_token='out_of_vocab')
tokenizer_encoder.fit_on_texts(new_data['input'].values)

tokenizer_decoder = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\]^_`{|}~\t\n')
tokenizer_decoder.fit_on_texts(np.append(new_data['decoder_input'].values,'<end>')) #to add  in the corpus

vocab_size_enc = len(tokenizer_encoder.word_index)+1
vocab_size_dec = len(tokenizer_decoder.word_index)+1

print("Vocab sizes of encoder and decoder are ",vocab_size_enc,vocab_size_dec)

Vocab sizes of encoder and decoder are  14156 10870


In [ ]:
vocab_size_enc = len(tokenizer_encoder.word_index)+1
vocab_size_dec = len(tokenizer_decoder.word_index)+1

In [ ]:
tokenizer_decoder.word_index['<end>']

10869

In [10]:
import pickle
outfile = open('/content/drive/MyDrive/tokenizers.pkl','wb')
pickle.dump((tokenizer_encoder,tokenizer_decoder),outfile)
outfile.close()

In [11]:
import pickle
infile = open('/content/drive/MyDrive/tokenizers.pkl','rb')
tokenizer_encoder,tokenizer_decoder = pickle.load(infile)
infile.close()

In [12]:
'''change the sentences to tokens and then pad them to maximum length such that all inputs and outputs'''

tokenzd_seq_enc = tokenizer_encoder.texts_to_sequences(new_data['input'].values)
tokenzd_seq_dec_ip = tokenizer_decoder.texts_to_sequences(new_data['decoder_input'].values)
tokenzd_seq_dec_op = tokenizer_decoder.texts_to_sequences(new_data['decoder_output'].values)

enc_padded = pad_sequences(tokenzd_seq_enc, maxlen=max_len_enc, dtype='int32', padding='post')
dec_ip_padded = pad_sequences(tokenzd_seq_dec_ip, maxlen=max_len_dec, dtype='int32', padding='post')
dec_op_padded = pad_sequences(tokenzd_seq_dec_op, maxlen=max_len_dec, dtype='int32', padding='post')

print(enc_padded.shape,dec_ip_padded.shape,dec_op_padded.shape)

(85441, 16) (85441, 5) (85441, 5)


In [ ]:
new_data['decoder_output'].values

array(['on any changes that <end>', 'any changes that <end>',
       'changes that <end>', ..., 'arty for netco <end>',
       'rty for netco <end>', 'ty for netco <end>'], dtype=object)

In [13]:
'''convert our data into tf.data format for easy and fast loading of batches'''

BATCH_SIZE=128
train_dataset = tf.data.Dataset.from_tensor_slices(((enc_padded, dec_ip_padded),dec_op_padded))
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
tokenzd_seq_dec_op

[[6, 27, 209, 34, 10892],
 [27, 209, 34, 10892],
 [209, 34, 10892],
 [34, 86, 3, 17, 10892],
 [86, 3, 17, 10892],
 [3, 17, 10892],
 [1967, 109, 167, 21, 10892],
 [109, 167, 21, 10892],
 [167, 21, 10892],
 [177, 658, 80, 15, 10892],
 [658, 80, 15, 10892],
 [80, 15, 10892],
 [15, 665, 520, 869, 10892],
 [665, 520, 869, 10892],
 [520, 869, 10892],
 [16, 356, 3, 3583, 10892],
 [356, 3, 3583, 10892],
 [3, 3583, 10892],
 [3583, 5687, 14, 684, 10892],
 [5687, 14, 684, 10892],
 [14, 684, 10892],
 [206, 7, 99, 46, 10892],
 [7, 99, 46, 10892],
 [99, 46, 10892],
 [46, 6745, 110, 980, 10892],
 [6745, 110, 980, 10892],
 [110, 980, 10892],
 [2985, 485, 5688, 563, 10892],
 [485, 5688, 563, 10892],
 [5688, 563, 10892],
 [2, 1495, 8, 2, 10892],
 [1495, 8, 2, 10892],
 [8, 2, 10892],
 [2, 716, 263, 2356, 10892],
 [716, 263, 2356, 10892],
 [263, 2356, 10892],
 [13, 1059, 535, 15, 10892],
 [1059, 535, 15, 10892],
 [535, 15, 10892],
 [15, 3584, 2, 716, 10892],
 [3584, 2, 716, 10892],
 [2, 716, 10892],
 [268

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2023-10-14 02:33:38--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-10-14 02:33:39--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-10-14 02:33:39--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
!unzip glove*.zip

Archive:  glove.6B.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of glove.6B.zip or
        glove.6B.zip.zip, and cannot find glove.6B.zip.ZIP, period.


In [14]:
'''Load pre trained word embeddings to encoder and decoder's embedding layers'''

embeddings_index = dict()
f = open('/content/drive/MyDrive/glove.6B.100d.txt')
for line in f:
 values = line.split()
 word = values[0]
 coefs = np.asarray(values[1:], dtype='float32')
 embeddings_index[word] = coefs
f.close()
embedding_matrix_enc = np.random.normal(size=(vocab_size_enc, 100))
for word, i in tokenizer_encoder.word_index.items():
 embedding_vector = embeddings_index.get(word)
 if embedding_vector is not None:
  embedding_matrix_enc[i] = embedding_vector

In [15]:
embeddings_index = dict()
f = open('/content/drive/MyDrive/glove.6B.100d.txt')
for line in f:
 values = line.split()
 word = values[0]
 coefs = np.asarray(values[1:], dtype='float32')
 embeddings_index[word] = coefs
f.close()
embedding_matrix_dec = np.random.normal(size=(vocab_size_dec, 100))
for word, i in tokenizer_decoder.word_index.items():
 embedding_vector = embeddings_index.get(word)
 if embedding_vector is not None:
  embedding_matrix_dec[i] = embedding_vector

In [16]:
class Encoder(tf.keras.layers.Layer):

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):

        '''Initialize the embedding and LSTM layers that will be used in our encoder layer'''

        super().__init__()
        self.lstm_size = lstm_size
        self.embedding = Embedding(input_dim=inp_vocab_size, output_dim=embedding_size, input_length=input_length,
                    mask_zero=True, name="embedding_layer_encoder", weights=[embedding_matrix_enc], trainable=True)
        # self.embedding = Embedding(input_dim=inp_vocab_size, output_dim=embedding_size, input_length=input_length,
        #                            mask_zero=True, name="embedding_layer_encoder")
        self.lstm = LSTM(lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")
        self.lstm_output = 0
        self.lstm_state_h=0
        self.lstm_state_c=0


    def call(self,input_sequence):

        '''
        Takes encoder's input sequence and return encoder's lstm's output
        '''

        input_embedd = self.embedding(input_sequence)
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd)
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c

In [17]:
class Decoder(tf.keras.layers.Layer):

    def __init__(self,out_vocab_size,embedding_size,lstm_size,input_length):

        '''Initialize the embedding and LSTM layers that will be used in our decoder layer'''

        super().__init__()
        # self.embedding = Embedding(input_dim=out_vocab_size, output_dim=embedding_size, input_length=input_length,
        #                    mask_zero=True, name="embedding_layer_decoder")
        self.embedding = Embedding(input_dim=out_vocab_size, output_dim=embedding_size, input_length=input_length,
                           mask_zero=True, name="embedding_layer_decoder", weights=[embedding_matrix_dec], trainable=True)
        self.lstm = LSTM(lstm_size, return_sequences=True, return_state=True, name="Decoder_LSTM")
        self.lstm_output = 0
        self.lstm_state_h=0
        self.lstm_state_c=0



    def call(self,input_sequence,initial_states):

        '''
        Takes decoder's input for teacher forcing, encoder's final states as initial states for decoder's lstm and return decoder lstm's output
        '''

        input_embedd = self.embedding(input_sequence)
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd, initial_state=initial_states)
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c

In [ ]:
'''Create a functional model with the custom layers built above and train it with our data'''

embed_dim = 100
enc_input_length = max_len_enc
dec_input_length = max_len_dec
lstm_units = 32

X_input1 = Input(shape=(max_len_enc,))
X_input2 = Input(shape=(max_len_dec,))
X_input3 = Input(shape=(max_len_dec,))

encoder_output, encoder_h, encoder_c = Encoder(inp_vocab_size=vocab_size_enc, embedding_size=embed_dim,input_length=enc_input_length, lstm_size = lstm_units)(X_input1)
decoder_output,_,_ = Decoder(out_vocab_size=vocab_size_dec, embedding_size=embed_dim,input_length=dec_input_length, lstm_size = lstm_units)(X_input2,initial_states=(encoder_h, encoder_c))
output = Dense(vocab_size_dec, activation='softmax')(decoder_output)

model = Model(inputs = [[X_input1,X_input2],X_input3], outputs = output)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy')
train_steps=new_data.shape[0]//BATCH_SIZE
#history = model.fit(train_dataset, steps_per_epoch=train_steps, epochs=5)

In [18]:
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/final_enc_dec_11')

In [ ]:
history = model.fit(train_dataset, steps_per_epoch=train_steps, epochs=30)

Epoch 1/30
667/667 [==============================] - 132s 187ms/step - loss: 5.9586
Epoch 2/30
667/667 [==============================] - 124s 186ms/step - loss: 5.2484
Epoch 3/30
667/667 [==============================] - 123s 184ms/step - loss: 5.0797
Epoch 4/30
667/667 [==============================] - 120s 179ms/step - loss: 4.9192
Epoch 5/30
667/667 [==============================] - 125s 188ms/step - loss: 4.7618
Epoch 6/30
667/667 [==============================] - 123s 184ms/step - loss: 4.6042
Epoch 7/30
667/667 [==============================] - 117s 176ms/step - loss: 4.4660
Epoch 8/30
667/667 [==============================] - 120s 180ms/step - loss: 4.3320
Epoch 9/30
667/667 [==============================] - 120s 179ms/step - loss: 4.2064
Epoch 10/30
667/667 [==============================] - 118s 176ms/step - loss: 4.0913
Epoch 11/30
667/667 [==============================] - 118s 176ms/step - loss: 3.9840
Epoch 12/30
667/667 [==============================] - 124s 186

In [ ]:
history = model.fit(train_dataset, steps_per_epoch=train_steps, epochs=15)

Epoch 1/15
667/667 [==============================] - 126s 188ms/step - loss: 2.5644
Epoch 2/15
667/667 [==============================] - 122s 183ms/step - loss: 2.5199
Epoch 3/15
667/667 [==============================] - 120s 181ms/step - loss: 2.4782
Epoch 4/15
667/667 [==============================] - 123s 185ms/step - loss: 2.4385
Epoch 5/15
667/667 [==============================] - 124s 185ms/step - loss: 2.4046
Epoch 6/15
667/667 [==============================] - 123s 184ms/step - loss: 2.3707
Epoch 7/15
667/667 [==============================] - 116s 173ms/step - loss: 2.3362
Epoch 8/15
667/667 [==============================] - 124s 186ms/step - loss: 2.3040
Epoch 9/15
667/667 [==============================] - 122s 183ms/step - loss: 2.2745
Epoch 10/15
667/667 [==============================] - 121s 182ms/step - loss: 2.2448
Epoch 11/15
667/667 [==============================] - 124s 186ms/step - loss: 2.2123
Epoch 12/15
667/667 [==============================] - 124s 185

In [ ]:
history = model.fit(train_dataset, steps_per_epoch=train_steps, epochs=15)

Epoch 1/15
667/667 [==============================] - 129s 194ms/step - loss: 2.0809
Epoch 2/15
667/667 [==============================] - 124s 186ms/step - loss: 2.0581
Epoch 3/15
667/667 [==============================] - 123s 184ms/step - loss: 2.0339
Epoch 4/15
667/667 [==============================] - 122s 183ms/step - loss: 2.0106
Epoch 5/15
667/667 [==============================] - 122s 182ms/step - loss: 1.9859
Epoch 6/15
667/667 [==============================] - 117s 176ms/step - loss: 1.9659
Epoch 7/15
667/667 [==============================] - 119s 178ms/step - loss: 1.9456
Epoch 8/15
667/667 [==============================] - 119s 178ms/step - loss: 1.9249
Epoch 9/15
667/667 [==============================] - 121s 181ms/step - loss: 1.9070
Epoch 10/15
667/667 [==============================] - 117s 176ms/step - loss: 1.8890
Epoch 11/15
667/667 [==============================] - 117s 176ms/step - loss: 1.8717
Epoch 12/15
667/667 [==============================] - 119s 179

In [ ]:
model.save('/content/drive/MyDrive/final_enc_dec_10')

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 16)]                 0         []                            
                                                                                                  
 input_5 (InputLayer)        [(None, 5)]                  0         []                            
                                                                                                  
 encoder_1 (Encoder)         ((None, 16, 32),             1432624   ['input_4[0][0]']             
                              (None, 32),                                                         
                              (None, 32))                                                         
                                                                                            

In [19]:

def predict(input_sentence,model):

  '''
  Given an input sentence, we need to predict the next words.
  Tokenize and pad the input sentence,
  Pass it to encoder's embedding layer and its output to encoder's LSTM layer.
  Pass its final states hidden and cell states as inputs to decoder's LSTM
  Before that, initialize  word and pass it to decoders embedding layer and then to LSTM layer,
  We run a loop till we get  word.
  We give predicted word at decoder's current step as input to its next step.
  '''

  encoder_test_tokens = tokenizer_encoder.texts_to_sequences([input_sentence])
  padded_encoder_input = pad_sequences(encoder_test_tokens, maxlen=16, dtype='float32', padding='post')
  encoder = model.layers[2]
  encoder_op, enc_h, enc_c = encoder(padded_encoder_input)
  decoder = model.layers[3]
  index_of_start = np.array(tokenizer_decoder.word_index['<start>']).reshape(1,1)
  states=(enc_h, enc_c)
  dense = model.layers[5]
  pred=0
  sentence = []
  while pred!=tokenizer_decoder.word_index['<end>']:
    dec_emb= decoder.embedding(index_of_start)
    predicted_out,state_h,state_c=decoder.lstm(dec_emb,states)
    pred = np.argmax(dense(predicted_out))
    word = [k for k in tokenizer_decoder.word_index if tokenizer_decoder.word_index[k]==pred][0]
    sentence.append(word)
    states= (state_h, state_c)
    index_of_start = np.array(pred).reshape(1,1)
  return ' '.join(sentence[:-1])

In [20]:
input_sentt = 'manti operating company should you have questio'

In [21]:
predict(input_sentt, loaded_model)

'ns please call'

In [22]:
input_sent = 'questions he returns ann'

In [24]:
predict(input_sent, loaded_model)

'er credit watch'

In [26]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
sample = new_data.sample(500)
import nltk.translate.bleu_score as bleu

blue_scores=[]
for n,i in enumerate(sample.index):
  predicted = predict(new_data.loc[i]['input'],loaded_model).split()
  original = [x for x in new_data.loc[i]['decoder_input'].split() if x!='<start>']
  blue_scores.append(bleu.sentence_bleu([original],predicted))
  if n in range(30):
    print('-- ip:',new_data.loc[i]['input'],'-- prd: ' ,' '.join(predicted))
    print('-- ip:',new_data.loc[i]['input'],'-- orgnl: ' ,' '.join(original),'\n')

np.mean(blue_scores)

-- ip: too weird htm get your free download of explor -- prd:  er at http
-- ip: too weird htm get your free download of explor -- orgnl:  er at http 

-- ip: larry berger monthly account. mtg. calendar entry appointment lar -- prd:  ry monthly staff
-- ip: larry berger monthly account. mtg. calendar entry appointment lar -- orgnl:  ry berger monthly 

-- ip: ecar load and weather data the temps are from cincinnati cleveland det -- prd:  roit and pittsburgh
-- ip: ecar load and weather data the temps are from cincinnati cleveland det -- orgnl:  roit and pittsburgh 

-- ip: billing review request com date time pm cst -- prd:  ng rd th
-- ip: billing review request com date time pm cst -- orgnl:  billing org 

-- ip: michael byrne real time trader don please advise -- prd:  ng the attached
-- ip: michael byrne real time trader don please advise -- orgnl:  of your interest in 

-- ip: dean laurent and joe errigo props to the tick -- prd:  ets and maple
-- ip: dean laurent and joe errigo p

0.006